<a href="https://colab.research.google.com/github/jiayixie/DSproject/blob/master/Shopify_Protect_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In terms of performance on Shopify Protect:
production model > new model > minfraud model

TODO:
* Look at the bucket size on the backtest data. How does that compare with the production observation?
* Is the new model's bucket size closer to the desired bucket size?


In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%%bigquery df --project sdp-stg-payments

-- reference, note from Nick: https://docs.google.com/document/d/1GdJC3rP_V1nbwaCrHgTHy5YStf6-8TmyP3X6rpv6QgU/edit#heading=h.vqhkqkengpht
-- Shopify Protect analysis for stacking model: https://github.com/Shopify/mode-repository/pull/17323/files

WITH
-- reconstruct the logic to get orders that are eligible for shopify protect
order_line_item_shipping_stats AS (
  SELECT
    order_id,
    COUNTIF(requires_shipping) = COUNT(*) AS all_line_items_require_shipping
  FROM `shopify-dw.raw_shopify.line_items`
  WHERE created_at >= TIMESTAMP('2023-05-01') AND created_at <= TIMESTAMP('2023-09-01')
  GROUP BY 1
),
eligible_txn AS (
  SELECT
    shop_id,
    order_id,
    MAX(IF(card_wallet_type IN ('shopify_pay'), True, False)) AS is_shop_pay_order,
  FROM `shopify-dw.merchant_sales.order_transactions_summary` summary
  WHERE
   order_transaction_created_at >= TIMESTAMP('2023-05-01')
   AND order_transaction_created_at <= TIMESTAMP('2023-09-01')
  AND is_shopify_payments
  AND order_transaction_kind IN ('authorization', 'sale', 'capture')
  AND order_transaction_status IN ('success', 'pending')
  AND (is_gift_card IS NULL OR NOT is_gift_card)
  AND payment_attempt_id NOT IN (175, 152)
  -- AND card_wallet_type IN ('shopify_pay')
  GROUP BY 1, 2
),
target_stats AS (
  SELECT
    pas.order_id,
    MIN(pas.order_created_at) AS order_created_at,
    MAX(is_fraudulent_chargeback) AS is_fraudulent_chargeback, -- at least one is fruadulent chargeback
    SUM(chargeback_amount_usd) AS chargeback_amount_usd,
    SUM(merchant_net_loss_usd) AS merchant_net_loss_usd,
    SUM(gmv_usd) AS gmv_amount_usd,
    -- gmv_amount_usd, chargeback_amount
    FROM
    (SELECT *
      FROM `sdp-prd-payments.intermediate.payment_attempt_summary_with_targets`
      WHERE order_transaction_created_at >= TIMESTAMP('2023-05-01')
      AND order_transaction_created_at <= TIMESTAMP('2023-09-01')
    ) pas
    LEFT JOIN
    `sdp-prd-payments.intermediate.chargeback_summary`
    USING (order_transaction_id)
    JOIN
    (SELECT
      order_id,
      order_transaction_id,
      gmv_usd
      FROM
      `shopify-dw.marts.payments_gmv_revenue_costs_summary`
      WHERE order_transaction_created_at >= TIMESTAMP('2023-05-01')
      AND order_transaction_created_at <= TIMESTAMP('2023-09-01')
    ) gmv
    USING (order_transaction_id)
  GROUP BY 1
),
sp_fulfillment_condition AS (
  SELECT
  order_id,
  is_fulfillment_requirement_met,
  FROM `sdp-prd-payments.intermediate.shopify_protect`
  WHERE order_created_at >= TIMESTAMP('2023-05-01')
    AND order_created_at <= TIMESTAMP('2023-09-01')
),
spabm_expanded AS (
  SELECT
    order_id,
    order_created_at,
    all_line_items_require_shipping,
    is_shop_pay_order,
    shop.country_code AS shop_country_code,
    IF(is_shop_pay_order AND shop.country_code='US', TRUE, FALSE) AS is_shop_pay_us_order,
    IF(shop.country_code='US' AND is_shop_pay_order AND all_line_items_require_shipping, True, False) AS meet_base_condition,
    chargeback_amount_usd AS last_chargeback_amount_usd,
    merchant_net_loss_usd,
    gmv_amount_usd,
    is_fraudulent_chargeback AS last_chargeback_is_fraudulent,
    is_fulfillment_requirement_met AS has_fulfillment_verification,
  FROM
    eligible_txn
    JOIN
    order_line_item_shipping_stats
    USING (order_id)
    JOIN `shopify-dw.accounts_and_administration.shop_profile_current` AS shop # shop_dimension
    USING (shop_id)
    JOIN target_stats
    USING (order_id)
    JOIN sp_fulfillment_condition
    USING (order_id)
),

-- backtest results on order grain
backtest_prod_model_stats AS (
    SELECT
  CAST(order_id AS INT64) AS order_id,
  MAX(base.score) AS base_model_score,
  MAX(stripe.score) AS stripe_model_score,
  MAX(
    CASE WHEN base.score>=0.08293911643996627 THEN 3
    WHEN base.score >=0.01804950057715174 THEN 2
    WHEN base.score >=0 THEN 1
    END
  ) AS base_model_recommendation_numeric,
  MAX(
    CASE WHEN stripe.score>=0.06342936418950507 THEN 3
    WHEN stripe.score >=0.012592794373631477 THEN 2
    WHEN stripe.score >=0 THEN 1
    END
  ) AS stripe_model_recommendation_numeric,
  FROM
    (
      SELECT order_id, order_transaction_id, predicted_fraud_status_numeric.scores[OFFSET(1)] AS score
      FROM
      `sdp-prd-payments.risk.predictions_2023_11_14T14_45_52_220Z_092_model_v2_1`
      -- WHERE split_type = 'backtest'
    ) AS base
  LEFT JOIN (
      SELECT order_transaction_id, predicted_fraud_status_numeric.scores[OFFSET(1)] AS score
      FROM `sdp-prd-payments.risk.predictions_2023_11_14T22_12_29_918Z_645_model_v2_1`
      -- WHERE split_type = 'backtest'
      WHERE stripe_risk_level IS NOT NULL
      ) AS stripe
  USING (order_transaction_id)
  GROUP BY 1
),

backtest_new_model_stats AS (
  SELECT
  CAST(order_id AS INT64) AS order_id,
  MAX(base.score) AS base_model_score,
  MAX(stripe.score) AS stripe_model_score,
  MAX(
    CASE WHEN base.score>= 0.08998370638638577 THEN 3
    WHEN base.score >= 0.016983764566481107 THEN 2
    WHEN base.score >=0 THEN 1
    END
  ) AS base_model_recommendation_numeric,
  MAX(
    CASE WHEN stripe.score>=0.0779494865417474 THEN 3
    WHEN stripe.score >= 0.013702599398791778 THEN 2
    WHEN stripe.score >=0 THEN 1
    END
  ) AS stripe_model_recommendation_numeric,
  FROM
    (
      SELECT order_transaction_id, predicted_fraud_status_numeric.scores[OFFSET(1)] AS score
      FROM
      `sdp-prd-payments.risk.predictions_2023_11_16T13_30_24_525Z_034`
      WHERE split_type = 'backtest'
    ) AS base
  LEFT JOIN (
      SELECT order_transaction_id,  predicted_fraud_status_numeric.scores[OFFSET(1)] AS score
      FROM `sdp-prd-payments.risk.predictions_2023_11_16T21_50_25_620Z_202`
      WHERE split_type = 'backtest'
      AND stripe_risk_level IS NOT NULL
      ) AS stripe
  USING (order_transaction_id)
  JOIN (
    SELECT
    order_id,
    CAST(order_transaction_id AS STRING) AS order_transaction_id
  FROM `shopify-dw.merchant_sales.order_transactions_summary`
  WHERE
    is_shopify_payments
    AND order_transaction_created_at  >= TIMESTAMP('2023-05-15')
    AND order_transaction_created_at  <= TIMESTAMP('2023-08-14')
  GROUP BY 1, 2 ) AS order_to_txn
  USING (order_transaction_id)
  GROUP BY 1
),

br_minfraud_model_stats AS (
  SELECT
  order_id,
  MAX (
    CASE WHEN base_model_recommendation = 'Accept' THEN 1
      WHEN base_model_recommendation = 'Investigate' THEN 2
      WHEN base_model_recommendation = 'Cancel' THEN 3 END ) AS base_model_recommendation_numeric,
  MAX (
    CASE WHEN stripe_model_recommendation = 'Accept' THEN 1
      WHEN stripe_model_recommendation = 'Investigate' THEN 2
      WHEN stripe_model_recommendation = 'Cancel' THEN 3 END ) AS stripe_model_recommendation_numeric,
  FROM `sdp-prd-payments.intermediate.bladerunner_risk_evaluation`
  WHERE evaluation_context = 'active_evaluator'
  AND model_version IN ('ccaa867ecb9e4757a52bc97c905ab39a', '0abbfa8961e24329aa9518d20d446347')
  AND event_timestamp >= TIMESTAMP('2023-05-01')
  AND event_timestamp <= TIMESTAMP('2023-09-01')
  GROUP BY 1
),

backtest_combined_stats AS (
  SELECT
  order_id,
  prod_model.base_model_score AS prod_model_base_score,
  prod_model.stripe_model_score AS prod_model_stripe_score,
  new_model.base_model_score AS new_model_base_score,
  new_model.stripe_model_score AS new_model_stripe_score,

  COALESCE(prod_model.stripe_model_recommendation_numeric, prod_model.base_model_recommendation_numeric) AS prod_model_recommendation_numeric,
  COALESCE(new_model.stripe_model_recommendation_numeric, new_model.base_model_recommendation_numeric) AS new_model_recommendation_numeric,
  COALESCE(minfraud_model.stripe_model_recommendation_numeric, minfraud_model.base_model_recommendation_numeric) AS minfraud_model_recommendation_numeric,
  FROM backtest_prod_model_stats AS prod_model
  JOIN backtest_new_model_stats AS new_model USING (order_id)
  JOIN br_minfraud_model_stats AS minfraud_model USING (order_id)
),

backtest_order_grain AS (
  SELECT
    order_id,
    prod_model_base_score, prod_model_stripe_score,
    new_model_base_score, new_model_stripe_score,
    prod_model_recommendation_numeric AS production_model_recommendation_numeric,
    new_model_recommendation_numeric,
    minfraud_model_recommendation_numeric,
  FROM backtest_combined_stats
),
orders_with_attributes AS (
SELECT spabm_expanded.*,
  IF (meet_base_condition AND production_model_recommendation_numeric = 1, TRUE, FALSE) AS is_protection_active_production_model,
  IF (meet_base_condition AND new_model_recommendation_numeric = 1, TRUE, FALSE) AS is_protection_active_new_model,
  IF (meet_base_condition AND minfraud_model_recommendation_numeric = 1, TRUE, FALSE) AS is_protection_active_minfraud_model,

  IF (meet_base_condition AND production_model_recommendation_numeric = 1
      AND has_fulfillment_verification, TRUE, FALSE ) AS is_protected_production_model,
  IF (meet_base_condition AND new_model_recommendation_numeric = 1
      AND has_fulfillment_verification, TRUE, FALSE ) AS is_protected_new_model,
  IF (meet_base_condition AND minfraud_model_recommendation_numeric = 1
      AND has_fulfillment_verification, TRUE, FALSE ) AS is_protected_minfraud_model,

  IF (meet_base_condition AND production_model_recommendation_numeric = 1
      AND has_fulfillment_verification AND last_chargeback_is_fraudulent, TRUE, FALSE ) AS is_covered_production_model,
  IF (meet_base_condition AND new_model_recommendation_numeric = 1
      AND has_fulfillment_verification AND last_chargeback_is_fraudulent, TRUE, FALSE ) AS is_covered_new_model,
  IF (meet_base_condition AND minfraud_model_recommendation_numeric = 1
      AND has_fulfillment_verification AND last_chargeback_is_fraudulent, TRUE, FALSE ) AS is_covered_minfraud_model,
FROM backtest_order_grain bt
JOIN spabm_expanded
  ON bt.order_id = spabm_expanded.order_id
)
SELECT
  DATE_TRUNC(order_created_at, WEEK) AS year_month,
  COUNTIF(is_shop_pay_us_order) AS n_shop_pay_us,
  COUNTIF(is_protection_active_production_model) AS n_protection_active_production_model,
  COUNTIF(is_protection_active_new_model) AS n_protection_active_new_model,
  COUNTIF(is_protection_active_minfraud_model) AS n_protection_active_minfraud_model,

  COUNTIF(is_protected_production_model) AS n_protected_production_model,
  COUNTIF(is_protected_new_model) AS n_protected_new_model,
  COUNTIF(is_protected_minfraud_model) AS n_protected_minfraud_model,

  COUNTIF(is_covered_production_model) AS n_covered_production_model,
  COUNTIF(is_covered_new_model) AS n_covered_new_model,
  COUNTIF(is_covered_minfraud_model) AS n_covered_minfraud_model,

  SUM(IF(is_shop_pay_us_order, gmv_amount_usd, 0)) AS gmv_shop_pay_us,
  SUM(IF(is_protection_active_production_model, gmv_amount_usd, 0)) AS gmv_protection_active_production_model,
  SUM(IF(is_protection_active_new_model, gmv_amount_usd, 0)) AS gmv_protection_active_new_model,
  SUM(IF(is_protection_active_minfraud_model, gmv_amount_usd, 0)) AS gmv_protection_active_minfraud_model,

  SUM(IF(is_protected_production_model, gmv_amount_usd, 0)) AS gmv_protected_production_model,
  SUM(IF(is_protected_new_model, gmv_amount_usd, 0)) AS gmv_protected_new_model,
  SUM(IF(is_protected_minfraud_model, gmv_amount_usd, 0)) AS gmv_protected_minfraud_model,

  SUM(IF(is_covered_production_model, gmv_amount_usd, 0)) AS gmv_covered_production_model,
  SUM(IF(is_covered_new_model, gmv_amount_usd, 0)) AS gmv_covered_new_model,
  SUM(IF(is_covered_minfraud_model, gmv_amount_usd, 0)) AS gmv_covered_minfraud_model,

  -- gross loss, upper limit of loss
  -- SUM(IF(is_covered_production_model, last_chargeback_amount_usd + COALESCE(shopify_dispute_fee_usd, 0), 0)) AS gmv_gross_loss_production_model,
  -- SUM(IF(is_covered_new_model, last_chargeback_amount_usd + COALESCE(shopify_dispute_fee_usd, 0), 0)) AS gmv_gross_loss_new_model,
  -- have to ignore shopify_dispute_fee_usd for now, because we do not have it in our table
  SUM(IF(is_covered_production_model, last_chargeback_amount_usd, 0)) AS gmv_gross_loss_production_model,
  SUM(IF(is_covered_new_model, last_chargeback_amount_usd, 0)) AS gmv_gross_loss_new_model,
  SUM(IF(is_covered_minfraud_model, last_chargeback_amount_usd, 0)) AS gmv_gross_loss_minfraud_model,

  -- net loss = gross loss - chargebacks that we/merchant win
  -- SUM(IF(is_covered_production_model, -1*net_loss_with_shopify_fee_usd, 0)) AS gmv_net_loss_production_model,
  -- SUM(IF(is_covered_new_model, -1*net_loss_with_shopify_fee_usd, 0)) AS gmv_net_loss_new_model,
  -- use merchant_net_loss_usd for now, it is probably not quite right, need to check how is that computed.
  SUM(IF(is_covered_production_model, -1*merchant_net_loss_usd, 0)) AS gmv_net_loss_production_model,
  SUM(IF(is_covered_new_model, -1*merchant_net_loss_usd, 0)) AS gmv_net_loss_new_model,
  SUM(IF(is_covered_minfraud_model, -1*merchant_net_loss_usd, 0)) AS gmv_net_loss_minfraud_model,

FROM orders_with_attributes
GROUP BY 1
ORDER BY 1


Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
df.columns


Index(['year_month', 'n_shop_pay_us', 'n_protection_active_production_model',
       'n_protection_active_new_model', 'n_protection_active_minfraud_model',
       'n_protected_production_model', 'n_protected_new_model',
       'n_protected_minfraud_model', 'n_covered_production_model',
       'n_covered_new_model', 'n_covered_minfraud_model', 'gmv_shop_pay_us',
       'gmv_protection_active_production_model',
       'gmv_protection_active_new_model',
       'gmv_protection_active_minfraud_model',
       'gmv_protected_production_model', 'gmv_protected_new_model',
       'gmv_protected_minfraud_model', 'gmv_covered_production_model',
       'gmv_covered_new_model', 'gmv_covered_minfraud_model',
       'gmv_gross_loss_production_model', 'gmv_gross_loss_new_model',
       'gmv_gross_loss_minfraud_model', 'gmv_net_loss_production_model',
       'gmv_net_loss_new_model', 'gmv_net_loss_minfraud_model'],
      dtype='object')

In [13]:
for col in df.columns:
  if 'gmv' in col:
    df[col]=df[col].astype(float)

In [14]:
# create an organized dataframe
cases = ['production_model', 'new_model', 'minfraud_model']
keys = ['gmv_protection_active','gmv_protected','gmv_gross_loss', 'gmv_net_loss', 'n_protection_active', 'n_protected', 'n_covered']

df_info = pd.DataFrame({'case':cases})
for key in keys:
  values=[]
  for case in cases:
    column_name = key+"_"+case
    value = df[column_name].sum()
    values.append(value)
  df_info[key] = values
df_info['loss_rate(bps)'] = df_info['gmv_net_loss']/df_info['gmv_protected']/0.0001

df_info

,case,gmv_protection_active,gmv_protected,gmv_gross_loss,gmv_net_loss,n_protection_active,n_protected,n_covered,loss_rate(bps)
0,production_model,2.561739e+09,1.838675e+09,1.534033e+06,1090509.46,46533222,35593596,13758,5.930954
1,new_model,2.562776e+09,1.839440e+09,1.552435e+06,1109875.10,46559215,35607943,13840,6.033766
2,minfraud_model,2.572809e+09,1.845511e+09,1.848824e+06,1348497.83,46666820,35684009,15203,7.306907


In [15]:
#df_info['loss_rate_informal(bps)'] = df_info['gmv_shopify_protect_loss']/df_info['gmv_protection_active']/0.0001
base_case = 'production_model'
for key in keys+['loss_rate(bps)']:
  base_value = df_info[df_info.case==base_case][key]
  df_info[f'{key}_diff%'] = df_info[key].apply(lambda x: (x-base_value)*100./base_value)
  df_info[f'{key}_delta'] = df_info[key].apply(lambda x: (x-base_value))


df_info[[key for key in df_info.keys() if 'diff%' in key and 'gmv' in key or key=='case' or key in ['loss_rate(bps)', 'loss_rate(bps)_diff%']]].round(2)

,case,loss_rate(bps),gmv_protection_active_diff%,gmv_protected_diff%,gmv_gross_loss_diff%,gmv_net_loss_diff%,loss_rate(bps)_diff%
0,production_model,5.93,0.00,0.00,0.00,0.00,0.00
1,new_model,6.03,0.04,0.04,1.20,1.78,1.73
2,minfraud_model,7.31,0.43,0.37,20.52,23.66,23.20


In [16]:
df_info[[key for key in df_info.keys() if 'diff%' in key and 'n_' in key and 'gmv' not in key or key=='case']].round(2)

,case,n_protection_active_diff%,n_protected_diff%,n_covered_diff%
0,production_model,0.00,0.00,0.0
1,new_model,0.06,0.04,0.6
2,minfraud_model,0.29,0.25,10.5


In [17]:
# Overview of all the gmv & order metrics, summed over all time period
df_info[[key for key in df_info.keys() if 'diff%' not in key and 'delta' not in key or key=='case']].round(2)

,case,gmv_protection_active,gmv_protected,gmv_gross_loss,gmv_net_loss,n_protection_active,n_protected,n_covered,loss_rate(bps)
0,production_model,2.561739e+09,1.838675e+09,1534033.07,1090509.46,46533222,35593596,13758,5.93
1,new_model,2.562776e+09,1.839440e+09,1552435.03,1109875.10,46559215,35607943,13840,6.03
2,minfraud_model,2.572809e+09,1.845511e+09,1848824.06,1348497.83,46666820,35684009,15203,7.31


In [18]:
#Change of metrics, from base to new. delta = new - base(production_model)
df_info[[key for key in df_info.keys() if 'delta' in key or key=='case']].round(2)

,case,gmv_protection_active_delta,gmv_protected_delta,gmv_gross_loss_delta,gmv_net_loss_delta,n_protection_active_delta,n_protected_delta,n_covered_delta,loss_rate(bps)_delta
0,production_model,0.00,0.00,0.00,0.00,0,0,0,0.00
1,new_model,1037051.16,765638.77,18401.95,19365.64,25993,14347,82,0.10
2,minfraud_model,11070584.39,6836621.32,314790.98,257988.37,133598,90413,1445,1.38


In [19]:
#Change of metrics, from base to new. delta = new - base(production_model)
df_info[[key for key in df_info.keys() if 'delta' in key or key=='case']].round(2)

,case,gmv_protection_active_delta,gmv_protected_delta,gmv_gross_loss_delta,gmv_net_loss_delta,n_protection_active_delta,n_protected_delta,n_covered_delta,loss_rate(bps)_delta
0,production_model,0.00,0.00,0.00,0.00,0,0,0,0.00
1,new_model,1037051.16,765638.77,18401.95,19365.64,25993,14347,82,0.10
2,minfraud_model,11070584.39,6836621.32,314790.98,257988.37,133598,90413,1445,1.38
